In [2]:
# Imports
from ortools.linear_solver import pywraplp


In [3]:
"""Solve a multiple knapsack problem using a MIP solver."""

def create_data_model():
    data = {}
    weights = [48, 30, 42, 36, 36, 48, 42, 42, 36, 24, 30, 30, 42, 36, 36]
    values = [10, 30, 25, 50, 35, 30, 15, 40, 30, 35, 45, 10, 20, 30, 25]
    data['weights'] = weights
    data['values'] = values
    data['items'] = list(range(len(weights)))
    data['num_items'] = len(weights)
    num_bins = 5
    data['bins'] = list(range(num_bins))
    data['bin_capacities'] = [100, 100, 100, 100, 100]
    return data


data = create_data_model()

# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

# Variables.
# x[i, j] = 1 if item i is packed in bin j.
x = {}
for i in data['items']:
	for j in data['bins']:
		x[(i, j)] = solver.IntVar(0, 1, 'x_%i_%i' % (i, j))

# Constraints.
# Each item is assigned to at most one bin.
for i in data['items']:
	solver.Add(sum(x[i, j] for j in data['bins']) <= 1)

# The amount packed in each bin cannot exceed its capacity.
for j in data['bins']:
	solver.Add(
		sum(x[(i, j)] * data['weights'][i]
			for i in data['items']) <= data['bin_capacities'][j])

# Objective.
# Maximize total value of packed items.
objective = solver.Objective()
for i in data['items']:
	for b in data['bins']:
		objective.SetCoefficient(x[i, j], data['values'][i])
objective.SetMaximization()

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
	print(f'Total packed value: {objective.Value()}')
	total_weight = 0
	for j in data['bins']:
		print(f'Bin {j}')
		bin_weight = 0
		bin_value = 0
		for i in data['items']:
			if x[i, b].solution_value() > 0:
				print(
					f"Item {i} weight: {data['weights'][i]} value: {data['values'][i]}"
				)
				bin_weight += data['weights'][i]
				bin_value += data['values'][i]
		print(f'Packed bin weight: {bin_weight}')
		print(f'Packed bin value: {bin_value}\n')
		total_weight += bin_weight
	print(f'Total packed weight: {total_weight}')
else:
	print('The problem does not have an optimal solution.')


Total packed value: 130.0
Bin 0
Item 3 weight: 36 value: 50
Item 9 weight: 24 value: 35
Item 10 weight: 30 value: 45
Packed bin weight: 90
Packed bin value: 130

Bin 1
Item 3 weight: 36 value: 50
Item 9 weight: 24 value: 35
Item 10 weight: 30 value: 45
Packed bin weight: 90
Packed bin value: 130

Bin 2
Item 3 weight: 36 value: 50
Item 9 weight: 24 value: 35
Item 10 weight: 30 value: 45
Packed bin weight: 90
Packed bin value: 130

Bin 3
Item 3 weight: 36 value: 50
Item 9 weight: 24 value: 35
Item 10 weight: 30 value: 45
Packed bin weight: 90
Packed bin value: 130

Bin 4
Item 3 weight: 36 value: 50
Item 9 weight: 24 value: 35
Item 10 weight: 30 value: 45
Packed bin weight: 90
Packed bin value: 130

Total packed weight: 450
